# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

In [24]:
import sympy as sp

#pn, qn, qnP1, qnM1 = sp.symbols('p_n, q_n, q_{n+1}, q_{n-1}', commutative=False)
#xn, xdn, xnP1, xdnP1 = sp.symbols('\chi_n', '\chi^{\dagger}_n', '\chi_{n+1}', '\chi^{\dagger}_{n+1}')
n=sp.symbols('n')
a=sp.symbols('a')
V=sp.Function('V')
p=sp.IndexedBase('p',commutative=False)
q=sp.IndexedBase('q',commutative=False)
x=sp.IndexedBase('\chi',commutative=False)
xd=sp.IndexedBase('\chi^{\dagger}',commutative=False)

In [29]:
bosonNI=sp.expand( p[n]**2/(2*a) + ((a/2)*(q[n+1]-q[n-1])/(2*a))**2 )
bosonI=sp.expand( (a/2)*V(q[n])**2 + a*V(q[n])*(q[n+1]-q[n-1])/(2*a) )
fermionNI=sp.expand( (xd[n]*x[n+1]+xd[n+1]*x[n])/(2*a) )
fermionI=sp.expand( (-1)**n*sp.diff(V(q[n]),q[n])*(xd[n]*x[n]-(1/2)) )

totHam=bosonNI+bosonI+fermionNI+fermionI

In [30]:
bosonNI+bosonI

a*V(q[n])**2/2 + V(q[n])*q[n + 1]/2 - V(q[n])*q[n - 1]/2 + q[n + 1]**2/16 - q[n + 1]*q[n - 1]/8 + q[n - 1]**2/16 + p[n]**2/(2*a)

In [31]:
fermionNI + fermionI

(-1)**n*Derivative(V(q[n]), q[n])*\chi[n]*\chi^{\dagger}[n] - 0.5*(-1)**n*Derivative(V(q[n]), q[n]) + \chi[n + 1]*\chi^{\dagger}[n]/(2*a) + \chi[n]*\chi^{\dagger}[n + 1]/(2*a)

In [32]:
N=3
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {q[-1]: q[N-1], q[N]: q[0]
                         }
elif bcType == 'dirichlet':
    boundaryConditions = {q[-1]: 0, q[N]: 0}

In [33]:
ham=0

for i in range(0,N):
    ham+=totHam.subs(n,i).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?
ham

a*V(q[0])**2/2 + a*V(q[1])**2/2 + a*V(q[2])**2/2 + V(q[0])*q[1]/2 - V(q[0])*q[2]/2 - V(q[1])*q[0]/2 + V(q[1])*q[2]/2 + V(q[2])*q[0]/2 - V(q[2])*q[1]/2 + Derivative(V(q[0]), q[0])*\chi[0]*\chi^{\dagger}[0] - 0.5*Derivative(V(q[0]), q[0]) - Derivative(V(q[1]), q[1])*\chi[1]*\chi^{\dagger}[1] + 0.5*Derivative(V(q[1]), q[1]) + Derivative(V(q[2]), q[2])*\chi[2]*\chi^{\dagger}[2] - 0.5*Derivative(V(q[2]), q[2]) + q[0]**2/8 - q[0]*q[1]/8 - q[0]*q[2]/8 + q[1]**2/8 - q[1]*q[2]/8 + q[2]**2/8 + \chi[0]*\chi^{\dagger}[1]/(2*a) + \chi[1]*\chi^{\dagger}[0]/(2*a) + \chi[1]*\chi^{\dagger}[2]/(2*a) + \chi[2]*\chi^{\dagger}[1]/(2*a) + \chi[2]*\chi^{\dagger}[3]/(2*a) + \chi[3]*\chi^{\dagger}[2]/(2*a) + p[0]**2/(2*a) + p[1]**2/(2*a) + p[2]**2/(2*a)

In [34]:
a = sp.IndexedBase('a',commutative=False)
adag = sp.IndexedBase('a^{\dagger}',commutative=False)

m=1

HOdofSubs = {}
for i in range(0,N):
    HOdofSubs[q[i]] = 0.5*sp.sqrt(2/m)*(a[i] + adag[i])
    HOdofSubs[p[i]] = complex(0,1)*sp.sqrt(2*m)*(adag[i] - a[i])/2
    
sp.expand(ham.subs(HOdofSubs))

a*V(0.707106781186548*a[0] + 0.707106781186548*a^{\dagger}[0])**2/2 + a*V(0.707106781186548*a[1] + 0.707106781186548*a^{\dagger}[1])**2/2 + a*V(0.707106781186548*a[2] + 0.707106781186548*a^{\dagger}[2])**2/2 + 0.353553390593274*V(0.707106781186548*a[0] + 0.707106781186548*a^{\dagger}[0])*a[1] - 0.353553390593274*V(0.707106781186548*a[0] + 0.707106781186548*a^{\dagger}[0])*a[2] + 0.353553390593274*V(0.707106781186548*a[0] + 0.707106781186548*a^{\dagger}[0])*a^{\dagger}[1] - 0.353553390593274*V(0.707106781186548*a[0] + 0.707106781186548*a^{\dagger}[0])*a^{\dagger}[2] - 0.353553390593274*V(0.707106781186548*a[1] + 0.707106781186548*a^{\dagger}[1])*a[0] + 0.353553390593274*V(0.707106781186548*a[1] + 0.707106781186548*a^{\dagger}[1])*a[2] - 0.353553390593274*V(0.707106781186548*a[1] + 0.707106781186548*a^{\dagger}[1])*a^{\dagger}[0] + 0.353553390593274*V(0.707106781186548*a[1] + 0.707106781186548*a^{\dagger}[1])*a^{\dagger}[2] + 0.353553390593274*V(0.707106781186548*a[2] + 0.707106781186548